# Laboratorio 5 - Parte 1

### Máquinas de Vectores de Soporte

### 2019-I

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

In [1]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")
%matplotlib inline

## Ejercicio 1

En este ejercicio deben resolver un problema de clasificación multi-clase usando una SVM. Teniendo en cuenta que la formulación original de la SVM sólo permite resolver problemas bi-clase, deben comparar los resultados obtenidos usando una estrategia **Uno vs Uno** con una estrategia **Uno vs El resto**.

Cargamos los datos:

In [3]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

Consutar el manejo de la librería sklearn para entrenar un modelos SVM en: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC. Para el caso multiclase la librería ya tiene implementada la estrategia **Uno vs Uno**, así que en este caso sólo deben llamar correctamente los métodos. Complete el siguiente código usando un clasificador basado en SVM:

In [4]:
import math
import numpy as np
from numpy import random
from sklearn import svm
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold


#Validamos el modelo

Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
vectorSupportP = np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
j = 0
for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    
    #Normalizamos los datos
    scaler = preprocessing.StandardScaler().fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    modelo = svm.SVC(C=1.000,kernel='linear')
    modelo.fit(Xtrain,Ytrain)
    vectorSupportP[j]=(np.sum(modelo.n_support_)/np.size(Xtrain,0))
    #Validación
    Ytrain_pred = modelo.predict(Xtrain)
    Yest = modelo.predict(Xtest)
    
    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
    EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
    j += 1
print('porcentaje de vectores',str(np.mean(vectorSupportP)))        
print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))

porcentaje de vectores 0.26203839770590365
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9707771243055832+-0.015485734683194906


In [5]:
kernels =['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']
Cs =[0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]
gamas =[0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1]

eficienciaVali = []
intervaloVali= []
PvectorSupport= []

for i in range(len(kernels)):
    
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    vectorSupportP = np.zeros(Folds)

    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if (gamas[i] == 0):            
            modelo = svm.SVC(C=Cs[i],gamma='auto',kernel=kernels[i])
        else:
            modelo = svm.SVC(C=Cs[i],gamma=gamas[i],kernel=kernels[i])
        
        modelo.fit(Xtrain,Ytrain)
        
        #Validación
        Ytrain_pred = modelo.predict(Xtrain)
        Yest = modelo.predict(Xtest)
        
        #Evaluamos las predicciones del modelo con los datos de test
        vectorSupportP[j]=(np.sum(modelo.n_support_)/np.size(Xtrain,0))
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        j += 1
    
    eficienciaVali.append(np.mean(EficienciaVal))
    intervaloVali.append(np.std(EficienciaVal))
    PvectorSupport.append(np.mean(vectorSupportP))

Realice los experimientos necesarios para llenar la siguiente tabla:

In [7]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
df_types["Eficiencia en validacion"]=eficienciaVali
df_types["Intervalo de confianza"]=intervaloVali
df_types["% de Vectores de Soporte"]=PvectorSupport
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [8]:
qgrid_widget.get_changed_df()

Eficiencia en validacion  Intervalo de confianza  \
Kernel C       gamma                                                     
lineal 0.001   0.00                   0.796001                0.053296   
       0.010   0.00                   0.970699                0.029265   
       0.100   0.00                   0.980446                0.016611   
       1.000   0.00                   0.970777                0.015486   
       10.000  0.00                   0.970777                0.015486   
       100.000 0.00                   0.970777                0.015486   
rbf    0.001   0.01                   0.315623                0.108064   
               0.10                   0.277914                0.042752   
               1.00                   0.254171                0.001717   
       0.010   0.01                   0.315623                0.108064   
               0.10                   0.277914                0.042752   
               1.00                   0.254171                0.001717   
       0.100   0.01                   0.832031                0.036448   
               0.10                   0.277914                0.042752   
               1.00                   0.254171                0.001717   
       1.000   0.01                   0.984659                0.011521   
               0.10                   0.808549                0.040707   
               1.00                   0.256964                0.006486   
       10.000  0.01                   0.990246                0.008317   
               0.10                   0.815471                0.037698   
               1.00                   0.258369                0.006026   
       100.000 0.01                   0.990246                0.008317   
               0.10                   0.815471                0.037698   
               1.00                   0.258369                0.006026   

                      % de Vectores de Soporte  
Kernel C       gamma                            
lineal 0.001   0.00                   1.000000  
       0.010   0.00                   0.608322  
       0.100   0.00                   0.286595  
       1.000   0.00                   0.262038  
       10.000  0.00                   0.262038  
       100.000 0.00                   0.262038  
rbf    0.001   0.01                   1.000000  
               0.10                   1.000000  
               1.00                   0.999539  
       0.010   0.01                   1.000000  
               0.10                   1.000000  
               1.00                   1.000000  
       0.100   0.01                   1.000000  
               0.10                   1.000000  
               1.00                   1.000000  
       1.000   0.01                   0.643049  
               0.10                   0.972674  
               1.00                   1.000000  
       10.000  0.01                   0.577322  
               0.10                   0.972213  
               1.00                   1.000000  
       100.000 0.01                   0.577322  
               0.10                   0.972213  
               1.00                   1.000000

## Ejercicio 2

Cree dos funciones, una para entrenar un conjunto de modelos bajo la estrategia Uno vs el resto, usando como clasificador base una SVM. La segunda función debe usar el conjunto de modelos entrenados, y clasificar un conjunto de muestras de validación.

In [96]:
def TrainSVM_OnevsRest(X,Y,g='auto',c=1,k='linear'):
    models =[] 
    iters = len(np.unique(Y))
    for i in range(iters):
        Yaux = np.copy(Y)
        Yaux[Y != i]=-1
        Yaux[Y == i]=1
        if (g == 0):            
            modelo = svm.SVC(C=c,gamma='auto',kernel=k)
        else:
            modelo = svm.SVC(C=c,gamma=g,kernel=k)
        modelo.fit(X,Yaux)       
        models.append(modelo)
    return models

def ValidaSVM_OnevsRest(models,X,Y):
    predictsMajor = np.zeros(np.size(X,0))
    Eficiencias = []
    iters = len(models)
    j = 0
    for xi in X:
        Ypredict = np.zeros(iters)
        for i in range(iters):
            xi = xi.reshape(1,np.size(X,1))
            Ypredict[i]= models[i].predict(xi)
            #Eficiencias.append(np.mean(Ypredict.ravel() == Y.ravel()))
            
        #index = np.where(Ypredict == 1)
        #temporal = index[0][0]
        print(Ypredict)
        #predictsMajor[j]=temporal
        j = j +1 
        #indexMaximo = np.argmax(Eficiencias)
    return predictsMajor.reshape(np.size(X,0),1)  



from sklearn.multiclass import OneVsRestClassifier


def TrainSVM_OnevsRest2(X,Y,g='auto',c=1,k='linear'):
    if (g == 0):            
            modelo = svm.SVC(C=c,gamma='auto',kernel=k)
    else:
            modelo = svm.SVC(C=c,gamma=g,kernel=k)
    return OneVsRestClassifier(modelo).fit(X, Y) 

def ValidaSVM_OnevsRest2(cosombo,X):
    return cosombo.predict(X) 


def TrainSVM_OnevsRest3(X,Y,g=0,c=1,k='linear'):
    modelos = []
    n = len(np.unique(Y))
    for i in range(n):
        Yaux = np.copy(Y)
        Yaux[Y != i]=-1
        Yaux[Y == i]=1
        if (g == 0):            
            modelo = svm.SVC(C=c,gamma='auto',kernel=k,probability=True)
        else:
            modelo = svm.SVC(C=c,gamma=g,kernel=k,probability=True)
        modelo.fit(X,Yaux)       
        modelos.append(modelo)
    return modelos    

def ValidaSVM_OnevsRest3(modelos,X):
    
    for xi in X:
        for m in modelos:
            xi = xi.reshape(1,np.size(X,1))
            probabilities = m.predict_proba(xi)
            
            print(probabilities)
    return ["a"]



## Ejercicio 3

Use las funciones definidas en el punto anterior para llevar a cabo la prueba de simulación con el mismo conjunto de datos del punto 1.

In [113]:
Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
vectorSupportP = np.zeros(Folds)

skf = StratifiedKFold(n_splits=Folds)
j = 0
for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    
    #Normalizamos los datos
    scaler = preprocessing.StandardScaler().fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    
    modelos = TrainSVM_OnevsRest2(X=Xtrain,Y=Ytrain)
    
    #Validación
    Ytrain_pred = ValidaSVM_OnevsRest2(modelos,Xtrain)
    Yest = ValidaSVM_OnevsRest2(modelos,Xtest)

    for a in range(4):
        vectorSupportP[j]=vectorSupportP[j]+ (np.sum(modelos.estimators_[a].n_support_))/(np.size(Xtrain,0))    
    
    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
    EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
    j += 1
print('porcentaje de vectores',str(np.mean(vectorSupportP)))
print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))

porcentaje de vectores 0.27779131908730303
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9763325940928522+-0.01837606981622538


Realice los experimientos necesarios para llenar la siguiente tabla:

In [99]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
df_types["Eficiencia en validacion"][3] = "0.97633"
df_types["Intervalo de confianza"][3] = "0.01837"
df_types["% de Vectores de Soporte"][3] = "0.2778"
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [36]:
qgrid_widget.get_changed_df()

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
lineal 0.001   0.00                                                    
       0.010   0.00                                                    
       0.100   0.00                                                    
       1.000   0.00                   0.97633                0.01837   
       10.000  0.00                                                    
       100.000 0.00                                                    
rbf    0.001   0.01                                                    
               0.10                                                    
               1.00                                                    
       0.010   0.01                                                    
               0.10                                                    
               1.00                                                    
       0.100   0.01                                                    
               0.10                                                    
               1.00                                                    
       1.000   0.01                                                    
               0.10                                                    
               1.00                                                    
       10.000  0.01                                                    
               0.10                                                    
               1.00                                                    
       100.000 0.01                                                    
               0.10                                                    
               1.00                                                    

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                            
       0.010   0.00                            
       0.100   0.00                            
       1.000   0.00                    0.2778  
       10.000  0.00                            
       100.000 0.00                            
rbf    0.001   0.01                            
               0.10                            
               1.00                            
       0.010   0.01                            
               0.10                            
               1.00                            
       0.100   0.01                            
               0.10                            
               1.00                            
       1.000   0.01                            
               0.10                            
               1.00                            
       10.000  0.01                            
               0.10                            
               1.00                            
       100.000 0.01                            
               0.10                            
               1.00

*En las tablas el punto separa las cifras decimales, es decir 100.000 es 100 ó 1.000 es 1.